
**KEY Server**



In [ ]:
!pip install pyDES

  Created wheel for pyDES: filename=pyDes-2.0.1-py2.py3-none-any.whl size=9548 sha256=ffd2998e6c210856ce8a53ba1741b0d9039252b1ad8da2a1fb44526179453cdd
  Stored in directory: /root/.cache/pip/wheels/8d/ff/d6/b38861c6a8fb5807fcd62e520d4d018caca96c8c3c645e47ec
Successfully built pyDES


In [ ]:
import os

print(os.urandom(8));
Ks=os.urandom(16)


b"\xb8\xc7n\x8a'\xd7K'"


In [ ]:
#key=b'\x13c\x8a\x1cg\x98\x1c\x89'
#print(type(key))

In [ ]:
from pyDes import *
import binascii
import hashlib
#key=Ks
def key_server(data):
  #data = "DES Algorithm Implementation"


  k = triple_des(Ks, CBC, pad=None, padmode=PAD_PKCS5)
  d = k.encrypt(data)
  Te=hashlib.md5(binascii.b2a_hex(d).decode(encoding='utf-8').encode())
  return  (Te.hexdigest())

  #print ("Encrypted: %r" % d)
  #print ("Decrypted: %r" % k.decrypt(d))

**Enter message**

In [ ]:
message=input("Enter message: ")

Enter message: hai


**Finding H(m) (SHA-256 equivalent for the message)**

In [ ]:
result = hashlib.sha256(message.encode()) 
HM=result.hexdigest()
print(result.hexdigest()) 

739b9d664d77db3ee3cab40737c6386fb7b6ef9e00b31a434fd7680e69482ba0


**Sending H(m) to the key server for generating common key K**

In [ ]:
K=key_server(HM)
print(K)

343e45f01a2f26e2ccb797e86d0395a0


**AES Encryption function**

In [ ]:
!pip install pycrypto

     |████████████████████████████████| 450kB 5.5MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499931 sha256=589686d627c4a8aabda6130f2f1643b0b43d830638f1c9534876ceb17f57f8cb
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built pycrypto


In [ ]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto import Random
import binascii

# AES supports multiple key sizes: 16 (AES128), 24 (AES192), or 32 (AES256).
key_bytes = 32

# Takes as input a 32-byte key and an arbitrary-length plaintext and returns a
# pair (iv, ciphtertext). "iv" stands for initialization vector.
iv = Random.new().read(AES.block_size)
def encrypt(key, plaintext):
    assert len(key) == key_bytes

    # Choose a random, 16-byte IV.
    

    # Convert the IV to a Python integer.
    iv_int = int(binascii.hexlify(iv), 16) 

    # Create a new Counter object with IV = iv_int.
    ctr = Counter.new(AES.block_size * 8, initial_value=iv_int)

    # Create AES-CTR cipher.
    aes = AES.new(key, AES.MODE_CTR, counter=ctr)

    # Encrypt and return IV and ciphertext.
    ciphertext = aes.encrypt(plaintext)
    return (iv, ciphertext)

# Takes as input a 32-byte key, a 16-byte IV, and a ciphertext, and outputs the
# corresponding plaintext.
def decrypt(key, iv, ciphertext):
    assert len(key) == key_bytes

    # Initialize counter for decryption. iv should be the same as the output of
    # encrypt().
    iv_int = int(binascii.hexlify(iv), 16) 
    ctr = Counter.new(AES.block_size * 8, initial_value=iv_int)

    # Create AES-CTR cipher.
    aes = AES.new(key, AES.MODE_CTR, counter=ctr)

    # Decrypt and return the plaintext.
    plaintext = aes.decrypt(ciphertext)
    return plaintext

In [ ]:
AES.block_size

16

**Encrypting Message with key K**

In [ ]:
ivm,C1=encrypt(K,message)

**generating private key Kx**

In [ ]:
Kx= Random.new().read(32)
print(Kx)

b'(\xf3P\xb7\xf3\x0e\x07;h\xe3c\x82\x94n\xb0\x8a\xa4\xe8\xec\x93!\xa0.\xfe6\xa5"\xd0!\x88\x1cR'


**Encrypting Common key K with private key Kx using AES encryption**


In [ ]:
ivk,C2=encrypt(Kx,K)

**Common cipher Text C1 and initial value**

In [ ]:
print("Common cipher Text C1: ", C1)

Common cipher Text C1:  b'!G\xfa'


In [ ]:
print("Initial value for message decryption: ", ivm)

Initial value for message decryption:  b'U\x9a\r:b\t\xde\xac\xe4\xf6s\x10\xf9\xa3\x01S'


**Encrypted Common key K (C2) and initial value for decryption**

In [ ]:
print("Encrypted Common key K (C2): ", C2)

Encrypted Common key K (C2):  b'\xe7\xb6\x94\x89\xea\xd0\xbci\x05Z/|M\xb5\xa8\xbd\x1bD\x13I\x85h\xb4\x95\xf5\xa6E\xde0\xc3V0'


In [ ]:
print("Initial value for K's decryption: ", ivk)

Initial value for K's decryption:  b'U\x9a\r:b\t\xde\xac\xe4\xf6s\x10\xf9\xa3\x01S'


**DECRYPTION**

In [ ]:
#private key (Kx)
#Kx=b"V@\xba`\x044\x12\xd2\xe6\xc3hD\xb3\xf7\xa5s\x956\xb59\xb2R\x12\xf0\xf9'\xbc\x805\x9b\xf3\x8d"
#ivm=
#C1=

**Decrypting Common key K**

In [ ]:
Common_key=decrypt(Kx, ivk, C2)

In [ ]:
Common_key

b'6add51858040cdc43459a2fa9b0f7bbe'

In [ ]:
Decrypted_msg=decrypt(Common_key,ivm,C1)

In [ ]:
print("Decrypted message: ",Decrypted_msg)

Decrypted message:  b'hai'


**Encryption**

In [ ]:
C1_list=[]
C2_list=[]

In [ ]:
message_t=input("Enter message: ")
result = hashlib.sha256(message.encode()) 
HM=result.hexdigest()
#print(result.hexdigest()) 
K=key_server(HM)
#print(K)


Enter message: hello hai


**Generate private key**

In [ ]:

Kx= Random.new().read(32)
print("This is your private key: ",Kx)


This is your private key:  b'\xd6\x1d\xfe`\xa3\x15\xc2zU\xc4J+\xa4\x86\xb2\x7f4\x85Bn\xaa\x8a \xd0\x8d\x96\x15\xf0v\x06\xb7\x9c'


In [ ]:
ivm,C1=encrypt(K,message_t)
ivk,C2=encrypt(Kx,K)
C1_list.append(C1)
C2_list.append(C2)

In [ ]:
print("Encrypted message: ", C1.hex())
print("Encrypted key: ", C2.hex())
print("Private key: ",Kx.hex())
print("Initial value for message: ", ivm.hex())
print("Initial value for common key: ",ivk.hex())

Encrypted message:  a6ebddc9783248ca24
Encrypted key:  b66d89f5d5e0eb09173c9a366b5530a8512fbbe2a4106fa880432bfef06212eb
Private key:  d61dfe60a315c27a55c44a2ba486b27f3485426eaa8a20d08d9615f07606b79c
Initial value for message:  de92e2b2cfdd6351bfb65ea992a85d68
Initial value for common key:  de92e2b2cfdd6351bfb65ea992a85d68


**Decryption**

In [ ]:
C1_t=bytes.fromhex(input("Enter message encrypted text: "))
C2_t=bytes.fromhex(input("Enter key encrypted text: "))
p_key=bytes.fromhex(input("Enter private key: "))
ivm_t=bytes.fromhex(input("Enter initial value for message"))
ivk_t=bytes.fromhex(input("Enter initial value for common key: "))


Enter message encrypted text: b18107
Enter key encrypted text: 85cbc0d5134330efac6c49ff5fa221c840bbe32f6a36bb900d55e0daad9c3353
Enter private key: 59e018145add3cbe1ed447cb49269cae36c46243d59e4cdc7e840d6a90d9acac
Enter initial value for message2df4589b7b34403323d0f0a4227f2f3b
Enter initial value for common key: 8f307c41fa5509ad32ccad83671136ee


In [ ]:
Common_key=decrypt(p_key, ivk_t, C2_t)
Decrypted_msg=decrypt(Common_key,ivm_t,C1_t)
print("Decrypted message: ",Decrypted_msg)


Decrypted message:  b'hai'


**Storage Details (For same message same Cipher text)**

In [ ]:
print("Encrypted messages")
for i in C1_list:
  print(i)

Encrypted messages
b'\xa6\xef\xd8'
b'\xa6\xef\xd8'
b'\xa6\xef\xd8'
b'\xa6\xeb\xdd\xc9x2H\xca$'


In [ ]:
print("Encrypted keys")
for i in C2_list:
  print(i)

Encrypted keys
b'&\xae\xdd\xdf\xce\x92\x84\x84U\xbb\x1aQU\xc5\xf8Y(]3\xfc\xde\xc1\xf5\xd7*\xad\xcb\n-\x18\xcaC'
b'\xe7\xc8\x87.\x96\xbb\xa8\x02m\xc4\xfb\xce\xe2)8\xa0\xc9\xa6\x96\xfb\xcb\xf7\x9a^g\x11,\xcfI\x83\x98`'
b'\x9fE)J\xf8\xbd\xa1`q\x8b\x8d\xae\xe3\xd8Q2\x1e$\x18\xebB\xb5\xff\x94\x82\x96I>\x04O\x9c!'
b'\xb6m\x89\xf5\xd5\xe0\xeb\t\x17<\x9a6kU0\xa8Q/\xbb\xe2\xa4\x10o\xa8\x80C+\xfe\xf0b\x12\xeb'


b"\x87\x98&V\x95\xb94'U\x8c*\xeeE\xf5\x97\xaa\n=d5\xa1E\x9bn\xf4<\x8cDO\x1e0/"